In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [9]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [10]:
# FewShotPromptTemplate

from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)
chain = prompt | chat
chain.invoke({"country": "Sweden"})

AI:
        I know this:
        Capital: Stockholm
        Language: Swedish
        Food: Meatballs and Herring
        Currency: Swedish Krona

AIMessageChunk(content='AI:\n        I know this:\n        Capital: Stockholm\n        Language: Swedish\n        Food: Meatballs and Herring\n        Currency: Swedish Krona')

In [11]:
# FewShotChatMessagePromptTemplate

from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "{answer}"),
    ]
)
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt, examples=examples
)
prompt = ChatPromptTemplate.from_messages(
    (
        [
            ("system", "You are a geography expert, you give short answers."),
            example_prompt,
            ("human", "{question}"),
        ]
    )
)
chain = prompt | chat
chain.invoke({"question": "Can you give me some information about Sweden?"})


        Sure! Here's what I know:
        Capital: Stockholm
        Language: Swedish
        Food: Meatballs and Herring
        Currency: Swedish Krona
        Notable Landmarks: Stockholm Archipelago, Vasa Museum, ABBA Museum

AIMessageChunk(content="\n        Sure! Here's what I know:\n        Capital: Stockholm\n        Language: Swedish\n        Food: Meatballs and Herring\n        Currency: Swedish Krona\n        Notable Landmarks: Stockholm Archipelago, Vasa Museum, ABBA Museum")